In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import chain
import ast
import statsmodels.api as sm

In [3]:
from linearmodels import PanelOLS

In [23]:
data = pd.read_csv("./Anna_scripts/asylum.csv")

In [273]:
data = pd.read_csv('df_final_final_ANNA.csv')

In [274]:
def indicatrice_sex(string):
    if string == 'M':
        return 1
    else : return 0
    
data['indicatrice_sex'] = data['sex'].apply(indicatrice_sex)

def mettre_sous_forme_liste(s):
    if s == 1 :
        return ['NA']
    lst = [elem for elem in s.split("'") if elem.strip()]
    lst.remove('[')
    lst.remove(']')
    return lst

data['Langues_citizen'] = data['Langues_citizen'].apply(mettre_sous_forme_liste)
data['Langues_geo'] = data['Langues_geo'].apply(mettre_sous_forme_liste)

data['Indicatrice_langue'] = 0

for i in tqdm(data.index) : 
    langue_citizen = data['Langues_citizen'][i]
    langue_geo = data['Langues_geo'][i]
    for ele in langue_citizen :
        for ele2 in langue_geo : 
            if ele == ele2:
                data.loc[i,'Indicatrice_langue']=1
                
#10000000 pour les coeff des rég ??
data['OBS_VALUE_POND'] = (data['OBS_VALUE'] / data['pop_citizen'])*10000000

data['gap_idh'] = data['idh_geo'] - data['idh_citizen']
data['gap_pib'] = data['pib_geo'] - data['pib_citizen']
data['gap_infla'] = data['infla_geo'] - data['infla_citizen']
data['gap_unemp'] = data['unemp_dest'] - data['unemp_origine']

data['Indicatrice_religion'] = 0

for i in tqdm(data.index) : 
    religion_citizen = data['religion_or'][i]
    religion_geo = data['religion_dest'][i]
    if religion_citizen == religion_geo:
        data.loc[i,'Indicatrice_religion']=1

data.to_csv('Data_for_reg_without0.csv', index=True)

100%|█████████████████████████████████████████████████████████████████████| 1297328/1297328 [00:51<00:00, 25048.28it/s]


In [3]:
data = pd.read_csv('Data_for_reg_without0.csv')

In [4]:
# Transformations : 
data['dist_carré'] = data['dist_capit'] ** 2
data['pib_geo'] = np.log(data['pib_geo'])

In [5]:
# Panel regression model
paneldata = data.set_index(['citizen', 'TIME_PERIOD']) #  panel data : organisa° : panel_id & time

y = (paneldata['OBS_VALUE'] / paneldata['pop_citizen']) * 1000000

x_geo = paneldata[['pib_geo', 'idh_geo', 
       'infla_geo','unemp_dest', 'pop_geo', 'fh_ep_dest',
       'fh_pr_dest', 'ipu_l_sw_dest', 'fh_fog_dest', 'dr_ig_dest',
        'wbgi_pve_dest']] #'ht_region_dest',
print(x_geo.shape)

x_geo_or = paneldata[['pib_geo', 'idh_geo', 
       'infla_geo','unemp_dest', 'pop_geo','fh_ep_dest',
       'fh_pr_dest', 'ipu_l_sw_dest', 'fh_fog_dest', 'dr_ig_dest',
       'wbgi_pve_dest', 'pib_citizen', 'idh_citizen', 'infla_citizen', 'unemp_origine', 'fh_status_origine',
       'fh_feb_origine', 'wbgi_cce_origine', 'fh_rol_origine', 'fh_cl_origine',
       'fh_pair_origine', 'wbgi_rle_origine', 
       'indicatrice_sex']] #wbgi_pve_origine
print(x_geo_or.shape)


x_geo_or_trans = paneldata[['pib_geo', 'idh_geo', 
       'infla_geo','unemp_dest', 'pop_geo', 'fh_ep_dest',
       'fh_pr_dest', 'ipu_l_sw_dest', 'fh_fog_dest', 'dr_ig_dest',
       'wbgi_pve_dest', 'pib_citizen', 'idh_citizen', 'infla_citizen', 'unemp_origine', 'fh_status_origine',
       'fh_feb_origine', 'wbgi_cce_origine', 'fh_rol_origine', 'fh_cl_origine',
       'fh_pair_origine', 'wbgi_rle_origine', 
       'indicatrice_sex', 'dist_capit','Indicatrice_langue', 'Indicatrice_religion']]
print(x_geo_or_trans.shape)

(1297328, 11)
(1297328, 23)
(1297328, 26)


In [7]:
from linearmodels import PanelOLS

In [8]:
model_geo = PanelOLS(y, x_geo, entity_effects=True, time_effects=True, drop_absorbed=True)
results_geo = model_geo.fit()

In [9]:
model_geo_or = PanelOLS(y, x_geo_or, entity_effects=True, time_effects=True, drop_absorbed=True)
results_geo_or = model_geo_or.fit()

In [10]:
model_geo_or_trans = PanelOLS(y, x_geo_or_trans, entity_effects=True, time_effects=True, drop_absorbed=True)
results_geo_or_trans = model_geo_or_trans.fit()

In [11]:
# Récupérer les noms de colonnes du dataframe
cols = ['pib_geo', 'idh_geo', 
       'infla_geo','unemp_dest', 'pop_geo', 'fh_ep_dest',
       'fh_pr_dest', 'ipu_l_sw_dest', 'fh_fog_dest', 'dr_ig_dest',
       'wbgi_pve_dest', 'pib_citizen', 'idh_citizen', 'infla_citizen', 'unemp_origine', 'fh_status_origine',
       'fh_feb_origine', 'wbgi_cce_origine', 'fh_rol_origine', 'fh_cl_origine',
       'fh_pair_origine', 'wbgi_rle_origine', 
       'indicatrice_sex', 'dist_capit','Indicatrice_langue', 'Indicatrice_religion']

# Initialiser le tableau final
df_final = pd.DataFrame(columns=cols)
df_final

for i in tqdm(df_final.columns):
    try : 
        df_final.loc['Destination', i] = results_geo.params[i]
        df_final.loc['Destination (p-val)', i] = results_geo.pvalues[i]
    except : None
    try : 
        df_final.loc['Destination \& Origine', i] = results_geo_or.params[i]
        df_final.loc['Destination \& Origine (p-val)', i] = results_geo_or.pvalues[i]
    except : None
    try : 
        df_final.loc['Destination, Origine \& Relations Bilatérales', i] = results_geo_or_trans.params[i]
        df_final.loc['Destination, Origine \& Relations Bilatérales (p-val)', i] = results_geo_or_trans.pvalues[i]
    except : None
        
df_final = df_final.fillna('')

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 56.18it/s]


In [12]:
df_final

,pib_geo,idh_geo,infla_geo,unemp_dest,pop_geo,fh_ep_dest,fh_pr_dest,ipu_l_sw_dest,fh_fog_dest,dr_ig_dest,...,fh_feb_origine,wbgi_cce_origine,fh_rol_origine,fh_cl_origine,fh_pair_origine,wbgi_rle_origine,indicatrice_sex,dist_capit,Indicatrice_langue,Indicatrice_religion
Destination,-0.043688,5.126654e+00,0.000329,-0.035313,3.877056e-08,-0.789395,-0.725844,0.149741,-0.554322,-0.595711,...,,,,,,,,,,
Destination (p-val),0.535807,3.219647e-14,0.961260,0.000000,0.000000e+00,0.002105,0.029707,0.530171,0.060965,0.299308,...,,,,,,,,,,
Destination \& Origine,-0.043701,5.118645e+00,0.000260,-0.035320,3.876936e-08,-0.770403,-0.499239,0.255968,0.444865,0.601601,...,-0.292844,0.402348,-0.171368,-0.121913,-3.330171,-1.885312,0.761399,,,
Destination \& Origine (p-val),0.535537,3.463896e-14,0.969313,0.000000,0.000000e+00,0.002983,0.170099,0.286402,0.159299,0.309216,...,0.394512,0.374257,0.669698,0.74273,0.0,0.001249,0.0,,,
"Destination, Origine \& Relations Bilatérales",-0.114835,5.462395e+00,0.002311,-0.034576,3.859988e-08,-0.770079,-0.499297,0.256616,0.442842,0.598377,...,-0.295083,0.40213,-0.1721,-0.126989,-3.325516,-1.891749,0.761381,-0.000005,0.418292,-0.018068
"Destination, Origine \& Relations Bilatérales (p-val)",0.110311,8.881784e-16,0.733020,0.000000,0.000000e+00,0.002995,0.170042,0.285176,0.161198,0.311808,...,0.3909,0.374509,0.668366,0.732415,0.0,0.001202,0.0,0.108539,0.0,0.793392


In [21]:
i=0
nb_col = int(input('nb col'))
print('\\begin{table}[h]')
lim = df_final.shape[1]
while i<lim+1 : 
    print("\\begin{tabular}{|"+"p{7cm}|"+"p{3.5cm}|"*(nb_col)+"}\n \\hline")
    print(" &", end=" ")
    for j in range(i, min(lim, i+nb_col) ):
        print("$"+df_final.columns[j]+"$", end=" ")
        if j != i+nb_col-1: 
            print("&", end=" ")
        else : print('\\\\')
    print("\\hline")
    for col in range(0,6,2) : 
        print(str(df_final.index[col+1]), end=" & ")
        for j in range(i, min(lim, i+ nb_col)) :
            if df_final.iloc[col,j] == '' : 
                print(str(df_final.iloc[col,j]), end=" ")
            else : print(str(round(df_final.iloc[col,j],4)), end=" ")
            if df_final.iloc[col,j] == '' : 
                print(str(df_final.iloc[col+1,j]), end=" ")
            else : print("("+str(round(df_final.iloc[col+1,j],4))+")", end=" ")
            if j != i+nb_col-1: 
                  print("&", end=" ")
            else : print('\\\\')
        print("\\hline")
    print("\end{tabular}")
    i = i+nb_col
print('\\caption{Résultats de la régression de panel}')
print('\\end{table}')

nb col4
\begin{table}[h]
\begin{tabular}{|p{7cm}|p{3.5cm}|p{3.5cm}|p{3.5cm}|p{3.5cm}|}
 \hline
 & $pib_geo$ & $idh_geo$ & $infla_geo$ & $unemp_dest$ \\
\hline
Destination (p-val) & -0.0437 (0.5358) & 5.1267 (0.0) & 0.0003 (0.9613) & -0.0353 (0.0) \\
\hline
Destination \& Origine (p-val) & -0.0437 (0.5355) & 5.1186 (0.0) & 0.0003 (0.9693) & -0.0353 (0.0) \\
\hline
Destination, Origine \& Relations Bilatérales (p-val) & -0.1148 (0.1103) & 5.4624 (0.0) & 0.0023 (0.733) & -0.0346 (0.0) \\
\hline
\end{tabular}
\begin{tabular}{|p{7cm}|p{3.5cm}|p{3.5cm}|p{3.5cm}|p{3.5cm}|}
 \hline
 & $pop_geo$ & $fh_ep_dest$ & $fh_pr_dest$ & $ipu_l_sw_dest$ \\
\hline
Destination (p-val) & 0.0 (0.0) & -0.7894 (0.0021) & -0.7258 (0.0297) & 0.1497 (0.5302) \\
\hline
Destination \& Origine (p-val) & 0.0 (0.0) & -0.7704 (0.003) & -0.4992 (0.1701) & 0.256 (0.2864) \\
\hline
Destination, Origine \& Relations Bilatérales (p-val) & 0.0 (0.0) & -0.7701 (0.003) & -0.4993 (0.17) & 0.2566 (0.2852) \\
\hline
\end{tabular}
